In [1]:
# Step 1: Install necessary libraries
!pip install requests pandas

# Step 2: Import libraries
import requests
import pandas as pd
import time
from google.colab import files





In [2]:
# 4 june

# TASK 1 ***************************************************************************************************************************************
*************************************************************

# 100 data pts - for year 20  - data downloaded using API KEY from TMDB website
# task1 -
#   fetch at least 100 movie ids from the tmdb using api's for the specific year i mentioned in the sheet

In [ ]:
# To modify the code to fetch data for movies released in 2017 and save only their IDs to a CSV file,
# we'll make some changes. Specifically,
# we'll use the TMDB API to search for movies released in 2017, collect their IDs, and then save those IDs to a CSV file.

In [3]:
import csv

In [6]:


# Step 3: Set your API key
api_key = '5abf4d126acc10d018d17b034dd7bb40'

# Step 4: Define a function to search for movies released in 2018
def search_movies_2018(page, api_key):
    url = f'https://api.themoviedb.org/3/discover/movie?api_key={api_key}&primary_release_year=2018&page={page}'
    response = requests.get(url)
    return response.json()

# Step 5: Define a function to collect 100 movie IDs for movies released in 2018
def collect_movie_ids_2018(api_key, num_movies=100):
    movie_ids = []
    page = 1
    total_pages = 1

    while len(movie_ids) < num_movies and page <= total_pages:
        try:
            data = search_movies_2018(page, api_key)
            for movie in data['results']:
                movie_ids.append(movie['id'])
                if len(movie_ids) >= num_movies:
                    break
            total_pages = data['total_pages']
            page += 1
        except Exception as e:
            print(f"Error fetching data for page {page}: {e}")
            page += 1

    return movie_ids[:num_movies]  # Ensure only the required number of movie IDs are returned

# Step 6: Collect the movie IDs for movies released in 2018
movie_ids_2018 = collect_movie_ids_2018(api_key, num_movies=100)




In [7]:
# Step 7: Convert the data to a DataFrame and save as CSV
df_movie_ids = pd.DataFrame(movie_ids_2018, columns=['movie_id'])
df_movie_ids.to_csv('tmdb_movie_ids_2018.csv', index=False)

# Step 8: Download the CSV file
files.download('tmdb_movie_ids_2018.csv')

print(f"Collected {len(movie_ids_2018)} movie IDs from 2018 and saved to tmdb_movie_ids_2018.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Collected 100 movie IDs from 2018 and saved to tmdb_movie_ids_2018.csv


In [ ]:
# 5 june task - 100 data pts - for 2018 year - for cols given below =

# task2-
#    fetch data with cols = id,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,keywords,cast,crew,revenue fields
#    using movie ids from the tmdb using api's for the specific year i mentioned in the sheet

In [1]:
import requests
import csv
import time
import pandas as pd
from google.colab import files

# Step 1: Set your API key
api_key = '5abf4d126acc10d018d17b034dd7bb40'

# Step 2: Define a function to search for movies released in 2018
def search_movies_2018(page, api_key):
    url = f'https://api.themoviedb.org/3/discover/movie?api_key={api_key}&primary_release_year=2018&page={page}'
    response = requests.get(url)
    return response.json()

# Step 3: Define a function to collect the first 200 movie IDs for movies released in 2018
def collect_movie_ids_2018(api_key, num_movies=200):
    movie_ids = []
    page = 1
    total_pages = 1

    while len(movie_ids) < num_movies and page <= total_pages:
        try:
            data = search_movies_2018(page, api_key)
            for movie in data['results']:
                movie_ids.append(movie['id'])
                if len(movie_ids) >= num_movies:
                    break
            total_pages = data['total_pages']
            page += 1
        except Exception as e:
            print(f"Error fetching data for page {page}: {e}")
            page += 1

    return movie_ids[:num_movies]  # Ensure only the required number of movie IDs are returned

# Step 4: Define a function to get detailed movie data from TMDB
def get_movie_details(movie_id, api_key):
    url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&append_to_response=keywords,credits'
    response = requests.get(url)
    if response.status_code == 429:  # Handle rate limit
        time.sleep(2)  # Wait a bit and retry
        response = requests.get(url)
    return response.json()

# Step 5: Define a function to collect detailed data for the collected movie IDs
def collect_movie_data(api_key, movie_ids, num_movies=100):
    movie_data = []
    count = 0

    for movie_id in movie_ids:
        try:
            data = get_movie_details(movie_id, api_key)
            if data.get('budget', 0) > 0 and data.get('revenue', 0) > 0:  # Ensure both budget and revenue are greater than 0
                movie_data.append({
                    'id': data.get('id'),
                    'budget': data.get('budget'),
                    'genres': ', '.join([genre['name'] for genre in data.get('genres', [])]),
                    'homepage': data.get('homepage'),
                    'imdb_id': data.get('imdb_id'),
                    'original_language': data.get('original_language'),
                    'original_title': data.get('original_title'),
                    'overview': data.get('overview'),
                    'popularity': data.get('popularity'),
                    'poster_path': data.get('poster_path'),
                    'production_companies': ', '.join([company['name'] for company in data.get('production_companies', [])]),
                    'production_countries': ', '.join([country['name'] for country in data.get('production_countries', [])]),
                    'release_date': data.get('release_date'),
                    'runtime': data.get('runtime'),
                    'spoken_languages': ', '.join([language['name'] for language in data.get('spoken_languages', [])]),
                    'status': data.get('status'),
                    'tagline': data.get('tagline'),
                    'title': data.get('title'),
                    'keywords': ', '.join([keyword['name'] for keyword in data.get('keywords', {}).get('keywords', [])]),
                    'cast': ', '.join([member['name'] for member in data.get('credits', {}).get('cast', [])]),
                    'crew': ', '.join([member['name'] for member in data.get('credits', {}).get('crew', [])]),
                    'revenue': data.get('revenue')
                })
                count += 1
                if count >= num_movies:
                    break
        except Exception as e:
            print(f"Error fetching details for movie ID {movie_id}: {e}")

    return movie_data

# Step 6: Collect the first 200 movie IDs for movies released in 2018
movie_ids_2018 = collect_movie_ids_2018(api_key, num_movies=200)

# Step 7: Collect the detailed movie data for movies with budget > 0 and revenue > 0
movie_data_2018 = collect_movie_data(api_key, movie_ids_2018, num_movies=100)

# movie_data_2018


In [17]:
# Step 8: Convert the data to a DataFrame and save as CSV
df_movie_data = pd.DataFrame(movie_data_2018)
df_movie_data.to_csv('tmdb_movie_data_2018.csv', index=False)

df_movie_data.head()

,id,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,...,release_date,runtime,spoken_languages,status,tagline,title,keywords,cast,crew,revenue
0,335983,116000000,"Science Fiction, Action",https://www.sonypictures.com/movies/venom,tt1270797,en,Venom,Investigative journalist Eddie Brock attempts ...,472.388,/2uNW4WbgBXL25BAbXGLnLqX71Sw.jpg,...,2018-09-28,112,"普通话, English, Bahasa melayu",Released,The world has enough Superheroes.,Venom,"spacecraft, san francisco, california, superhe...","Tom Hardy, Michelle Williams, Riz Ahmed, Scott...","Scott Rosenberg, Jeff Pinkner, David Michelini...",856085151
1,284054,200000000,"Action, Adventure, Science Fiction",https://www.marvel.com/movies/black-panther,tt1825683,en,Black Panther,"King T'Challa returns home to the reclusive, t...",382.579,/uxzzxijgPIY7slzFvMotPv8wjKA.jpg,...,2018-02-13,135,"English, 한국어/조선말, Kiswahili,",Released,Long live the king.,Black Panther,"africa, superhero, based on comic, aftercredit...","Chadwick Boseman, Michael B. Jordan, Lupita Ny...","Joseph Hiura, Kevin Feige, Christine Mahuna, J...",1349926083
2,395990,30000000,"Action, Crime, Drama, Thriller",https://www.deathwish.movie/,tt1137450,en,Death Wish,A mild-mannered father is transformed into a k...,343.839,/g0eEEZAqMf98ULQnoiORuazjOpn.jpg,...,2018-03-02,107,English,Released,They came for his family. Now he's coming for ...,Death Wish,"robbery, based on novel or book, police, gun, ...","Bruce Willis, Vincent D'Onofrio, Dean Norris, ...","Brian Garfield, Eli Roth, Joe Carnahan, Roger ...",34017028
3,490132,23000000,"Drama, History",http://GreenBookFilm.com,tt6966692,en,Green Book,"Tony Lip, a bouncer in 1962, is hired to drive...",372.343,/7BsvSuDQuoqhWmU2fL7W2GOcZHU.jpg,...,2018-11-16,130,"English, Italiano, Pусский",Released,Inspired by a True Friendship,Green Book,"friendship, southern usa, road trip, racism, p...","Viggo Mortensen, Mahershala Ali, Linda Cardell...","Peter Farrelly, Patrick J. Don Vito, Sean Port...",319700000
4,297802,160000000,"Action, Adventure, Fantasy",https://www.warnerbros.com/movies/aquaman,tt1477834,en,Aquaman,"Half-human, half-Atlantean Arthur Curry is tak...",307.462,/zdw7Wf97vsQ0YnGomxDqfcEdUjX.jpg,...,2018-12-07,143,English,Released,Home is calling.,Aquaman,"hero, chase, atlantis, superhero, half-brother...","Jason Momoa, Amber Heard, Willem Dafoe, Patric...","James Wan, Zack Snyder, Deborah Snyder, Rob Co...",1152028393


In [18]:
df_movie_data.shape

(100, 22)

In [19]:
df_movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    100 non-null    int64  
 1   budget                100 non-null    int64  
 2   genres                100 non-null    object 
 3   homepage              100 non-null    object 
 4   imdb_id               100 non-null    object 
 5   original_language     100 non-null    object 
 6   original_title        100 non-null    object 
 7   overview              100 non-null    object 
 8   popularity            100 non-null    float64
 9   poster_path           100 non-null    object 
 10  production_companies  100 non-null    object 
 11  production_countries  100 non-null    object 
 12  release_date          100 non-null    object 
 13  runtime               100 non-null    int64  
 14  spoken_languages      100 non-null    object 
 15  status                10

In [20]:
# Step 9: Download the CSV file
files.download('tmdb_movie_data_2018.csv')

print(f"Collected data for {len(movie_data_2018)} movies from 2018 with budget > 0 and saved to tmdb_movie_data_2018.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Collected data for 100 movies from 2018 with budget > 0 and saved to tmdb_movie_data_2018.csv
